In [1]:
import pandas as pd
import scipy
import scipy.stats as stats
import sys
import os
import numpy 
import pylab as pl
import json
import os
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#url to capture data
url="http://cosmo.nyu.edu/~fb55/UI_CUSP_2015/data/"

#dictionaries to hold values for males and females
allmales={}
allfemales={}

#import male data
allmales['all']=pd.read_excel(url+'/pinc08_2_1_1.xls',header=9)
allmales['white']=pd.read_excel(url+'/pinc08_2_1_4.xls', header=9)
allmales['black']=pd.read_excel(url+'/pinc08_2_1_6.xls', header=9)
allmales['hispanic']=pd.read_excel(url+'/pinc08_2_1_9.xls', header=9)
allmales['asian']=pd.read_excel(url+'/pinc08_2_1_8.xls', header=9)

#import female data
allfemales['all']=pd.read_excel(url+'/pinc08_3_1_1.xls', header=9)
allfemales['white']=pd.read_excel(url+'/pinc08_3_1_4.xls', header=9)
allfemales['black']=pd.read_excel(url+'/pinc08_3_1_6.xls', header=9)
allfemales['hispanic']=pd.read_excel(url+'/pinc08_3_1_9.xls', header=9)
allfemales['asian']=pd.read_excel(url+'/pinc08_3_1_8.xls', header=9)

In [3]:
#remove non-real entries for males
for k in allmales:
    nonreal  = allmales[k].applymap(np.isreal)
    for c in allmales[k].columns :
        if nonreal.sum()[c]>1 and nonreal.sum()[c]<49:
            allmales[k][c][~nonreal[c]] = np.float('NaN')
            allmales[k][c] = allmales[k][c].astype(float)

/Users/jgrundy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
#remove non-real entries for females
for k in allfemales:
    nonreal  = allfemales[k].applymap(np.isreal)
    for c in allfemales[k].columns :
        if nonreal.sum()[c]>1 and nonreal.sum()[c]<49:
            allfemales[k][c][~nonreal[c]] = np.float('NaN')
            allfemales[k][c] = allfemales[k][c].astype(float)

/Users/jgrundy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
for x in allmales:
    allmales[x] = allmales[x].convert_objects(convert_numeric=True)
    
for x in allfemales:
    allfemales[x] = allfemales[x].convert_objects(convert_numeric=True)

#allmales = allmales.ix[1:38].set_index(u'Characteristic')
#allfemales = allfemales.ix[1:38].set_index(u'Characteristic')

/Users/jgrundy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app
/Users/jgrundy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [32]:
racedata  = []

for x in allmales:
    maleincome = allmales[x]['Median income']

for x in allfemales:
    femaleincome = allfemales[x]['Median income']

49
   level_0  index  m_income  f_income   race
0        0      0       NaN       NaN  asian
1        1      1     40153     24840  asian


In [7]:
#plot
fig, ax = plt.subplots(figsize=(10,10))
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

ax.plot(maleincome, femaleincome)

ax.set_ylabel('Female income')
ax.set_xlabel('Male income')
ax.set_title('Male vs Female income')

ax.plot([0, 40000], [0, 40000], color='r', linestyle='--', linewidth=.6, label='equality axix')

NameError: name 'total' is not defined

In [ ]:
#this is the analytical best regression line. it can be proved (ask me about it if you want!)
cor = stats.spearmanr(total['maleincome'], total['femaleincome'])

beta1=cor[0]*total['femaleincome'].std()/total['maleincome'].std()
beta0=total['femaleincome'].mean()-beta1*total['maleincome'].mean()

print beta1
print beta0

In [35]:
all_males=np.ma.array(np.concatenate([allmales['all']['Median income'][:].values,allmales['white']['Median income'][:].values,allmales['black']['Median income'][:].values,allmales['hispanic']['Median income'][:].values,allmales['asian']['Median income'][:].values]).flatten())

all_females=np.ma.array(np.concatenate([allfemales['all']['Median income'][:].values,allfemales['white']['Median income'][:].values,allfemales['black']['Median income'][:].values,allfemales['hispanic']['Median income'][:].values,allfemales['asian']['Median income'][:].values]).flatten())

#I am gonna create the mask int he next line. 
#the mask is True where either male or female value is Nan. 
#Notice that True is really just >1, and False really 0. 
#so i can add the male and female mask and the resulting mask will be 
#False where both arrays are not Nan, and true otherwise. 

allmask = [np.isnan(all_males) + np.isnan(all_females)]

all_females.mask = allmask
all_males.mask = allmask

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

for name, group in groups:
    ax.plot(group.m_income, group.f_income, marker='o',  linestyle='', ms=6, alpha=.6, label=name )

ax.set_ylabel('Female income')
ax.set_xlabel('Male income')
ax.set_title('Male vs Female income')

ax.plot([0, 40000], [0, 40000], color='r', linestyle='--', linewidth=.6,  label='equality') ## equality axis
ax.plot(X, lm.predict(X), color='blue',linestyle='-', linewidth=.3, label='regression') ## least square model
ax.plot([0, 50000], [beta0, beta0+ 50000*beta1], color='g', linestyle='-', linewidth=1, label='analytical') ## analytical
ax.legend()
ax.legend(numpoints=1);

In [ ]:
For a given stipend, males are going to make approximately 62% of wa